# Visualization

In this notebook, we display the accuracy of our estimates on a map. For that, we use the results of the evaluation computed using the pipeline's output and the RNI. 

In [ ]:
import pandas as pd
import numpy as np
import os
import geojson
import folium
import json
import plotly.express as px

In [2]:
# helper function : computes the overlay that will be displayed on the Folium map.

def compute_geojson_overlay(results, communes):
    """
    returns a geojson with the coordinates of the cities 
    and some of their properties. 
    
    args : 
    - results : a datafame of results
    - communes_list : a list of json files corresponding to the cities of the targeted departements.
    """
    
    # retrieve the polygons of the commune

    target_communes = np.unique(results['city'].values)

    features = []

    for commune in communes.keys():
        for target in target_communes:

                if int(target) == int(communes[commune]['properties']['code_insee']):

                    polygon = communes[commune]['coordinates']

                    polygon = geojson.Polygon(polygon)  

                    ape = results[results['city'] == target]['APE'].values.item()

                    properties = {
                        'city' : int(target),
                        'APE' : ape,
                        'Nom' : communes[commune]['properties']['nom_commune']
                    }

                    features.append(geojson.Feature(geometry=polygon, properties=properties))

    feature_collection = geojson.FeatureCollection(features)
    
    return feature_collection

In [3]:
# target departement
dpt = 74

# directories, should match those contained in the config.yml file
directory = '../evaluation'
aux_dir = '../aux'
filename = "results_{}.csv".format(dpt)
# center location : the center of the map to be displayed

center = [45.818701443200275, 4.804099378122765]

In [4]:
# load the files and prepare the communes_list

# results dataframe
results = pd.read_csv(os.path.join(directory, filename))

# communes coordinates
# retrieve the communes file
communes = json.load(open(os.path.join(aux_dir, 'communes_{}.json'.format(dpt))))
    
# compute the geojson that will be displayed on the map
feature_collection = compute_geojson_overlay(results, communes)

In [8]:
import plotly.express as px
import plotly.graph_objects as go


dataframes = []

# from the data directory
for dpt in [1, 26, 38, 42, 69]:
    results = pd.read_csv(os.path.join('../evaluation','results_{}.csv'.format(dpt)))
    dataframes.append(results)
    
dataframes.append(pd.read_csv(os.path.join('../analysis/building/results_buildings.csv')))    
    
# append all dataframes
df = pd.concat(dataframes)


df = df[df['APE'] < 100]
    
fig1 = px.line(df, x="target_kWp", y="target_kWp")

fig2 = px.scatter(df, x="est_kWp", y="target_kWp", color="APE")

fig3 = go.Figure(data=fig1.data + fig2.data)

fig3.show()

## An example : plot the city-wise average percentage error (APE)

We first display the resuls on a map. The bins are manually set up.

In [ ]:
# bins : to be manually set up for a better display.
bins = [0, 15, 75, 100, 150, int(results['APE'].max()) + 1] 

m = folium.Map(location=center, zoom_start=9)

folium.Choropleth(
    geo_data = feature_collection,#geojson.load(open("geodata_results.geojson")),                  #json
    name ='APE',  
    bins = bins,
    data = results,                     
    columns = ['city', 'APE'], #columns to work on
    key_on ='properties.city',
    fill_color ='YlOrBr',     #I passed colors Yellow,Green,Blue
    fill_opacity = 0.7,
    line_opacity = 0.2,
legend_name = "APE"
).add_to(m)

style_dict = {
        "fillColor": "#ffff00",
        "weight": 0,
    }
# add an overlay with the commune properties (name, postal code and APE value)
geo_json = folium.GeoJson(data=feature_collection, popup=folium.GeoJsonPopup(fields=['Nom', 'city', 'APE']), style_function= lambda feature: style_dict)
geo_json.add_to(m)

m

Additionnally, we display the scatter plot of the APE. Each dot is a city, we plot the estimated installed capacity against the true installed capacity. 

In [ ]:
mean = results['APE'].mean()
print('Mean APE for the departement {} : {:0.2f} %'.format(dpt, mean))
px.scatter(results, 'est_kWp', 'target_kWp', color = 'APE', hover_data = ['city', 'target_count', 'est_count', 'APE'])

In [ ]:
directory = '../evaluation'
aux_dir = '../aux'

m = folium.Map(location=center, zoom_start=9)


for dpt in [1, 69, 38, 26, 42, 74]:
    communes = json.load(open(os.path.join(aux_dir, 'communes_{}.json'.format(dpt))))
    filename = "results_{}.csv".format(dpt)
    results = pd.read_csv(os.path.join(directory, filename))   
    
    print('MAPE in terms of estimated installed capacity for departement {} : {:.02f}'.format(dpt, results['APE'].mean()))
    
    feature_collection = compute_geojson_overlay(results, communes)
    
    # bins : to be manually set up for a better display.
    bins = [0, 15, 75, 100, 150, int(results['APE'].max()) + 1] 

    folium.Choropleth(
        geo_data = feature_collection,#geojson.load(open("geodata_results.geojson")),                  #json
        name ='APE',  
        bins = bins,
        data = results,                     
        columns = ['city', 'APE'], #columns to work on
        key_on ='properties.city',
        fill_color ='YlOrBr',     #I passed colors Yellow,Green,Blue
        fill_opacity = 0.7,
        line_opacity = 0.2,
    legend_name = "Departement {}".format(dpt)
    ).add_to(m)
    
    style_dict = {
            "fillColor": "#ffff00",
            "weight": 0,
        }
    # add an overlay with the commune properties (name, postal code and APE value)
    geo_json = folium.GeoJson(data=feature_collection, popup=folium.GeoJsonPopup(fields=['Nom', 'city', 'APE']), style_function= lambda feature: style_dict)
    geo_json.add_to(m)
    
    

m

In [ ]:
#m.save('map.html')